Grid search to determine best hyperparameters

Hyperparameters tuned: Number of filters, Filter size, # of Dense nodes

Best model: 16 filter, Size 4, 16 Dense nodes

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import random
import tensorflow as tf
import tensorflow.keras.layers as tfl
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, LambdaCallback
from tensorflow.keras.models import Sequential, load_model, Model
from scipy.signal import find_peaks

In [3]:
#Recieves data and applies z-score standardisation on all channels
def standardise(stored_data):
    scaler = StandardScaler()
    standard_stored_data = scaler.fit_transform(stored_data)
    return standard_stored_data

In [4]:
#Following code was made by adapting original code by Wen et. al (2021)
#Provided in their paper: "A convolutional neural network to identify motor units
#from high-density surface electromyography signals inreal time"
#Original code can be found here: https://github.com/ywen3/dcnn_mu_decomp/blob/main/hdEMG_DCNN.ipynb

#Used for calculating RoA during model training

def RoA_m(y_true, y_pred):
    threshold = 3*tf.math.reduce_std(y_pred)
    y_pred_binary = tf.where(y_pred>=threshold, 1., 0.)
    y_comp = y_pred_binary + y_true
    true_positives = tf.shape(tf.where(y_comp == 2))[0]
    unmatched = tf.shape(tf.where(y_comp == 1))[0]
    return true_positives/(true_positives + unmatched)


class AccuracyCallback(Callback):
    def __init__(self, metric_name = 'accuracy'):
        super().__init__()
        self.metric_name = metric_name
        self.val_metric = []
        self.metric = []
        self.val_metric_mean = 0
        self.metric_mean = 0
        self.best_metric = 0
        
    def on_epoch_end(self, epoch, logs=None):
#         print('Accuracycallback')
        # extract values from logs
        self.val_metric = []
        self.metric = []
        for log_name, log_value in logs.items():
            if log_name.find(self.metric_name) != -1:
                if log_name.find('val') != -1:
                    self.val_metric.append(log_value)
                else:
                    self.metric.append(log_value)

        self.val_metric_mean = np.mean(self.val_metric)
        self.metric_mean = np.mean(self.metric)
        logs['val_{}'.format(self.metric_name)] = np.mean(self.val_metric)   # replace it with your metrics
        logs['{}'.format(self.metric_name)] = np.mean(self.metric)   # replace it with your metrics

In [5]:
#Calculate the RoA given model predictions and true labels Y
def singleModelRoA(predictions, Y):
    y_pred = tf.squeeze(predictions)
    threshold = 3*np.std(y_pred,axis = 1)
    match = 0
    unmatch = 0
    for MU in range(len(Y)):
        pred_spikes, _ = find_peaks(y_pred[MU], height = threshold[MU], distance = 2)
        true_spikes = tf.squeeze(tf.where(np.array(Y)[MU] == 1))
        a = set(true_spikes.numpy())
        b = set(pred_spikes)
        matches = len(a.intersection(b))
        unmatched1 = a - b
        unmatched2 = b - a
        tolerance = len([x for x in unmatched1 if (x+1 in unmatched2 or x-1 in unmatched2)])
        match = match + matches + tolerance
        unmatch = unmatch + len(unmatched1) + len(unmatched2) - (2*tolerance)
    return match/(match + unmatch)

In [6]:
#Function for standardising and windowing training signal
def windowtrain(EMGtrain, spiketrain, window_size):
    
    EMGtrain = standardise(EMGtrain)
    x_train = []
    y_train = []
    for i in range(30,EMGtrain.shape[0]-120):
        
        if any(spiketrain[i, 0:5] == 1):
            x_train.append(EMGtrain[i-10:i+(window_size-10),:])
            y_train.append(spiketrain[i, 0:5])
        else:
            if random.uniform(0, 1) < .05:
                x_train.append(EMGtrain[i-10:i+(window_size-10),:])
                y_train.append(spiketrain[i, 0:5])
    y_train = np.array(y_train)
    y_train2 = []
    for i in range(5):
        y_train2.append(y_train[:,i])
    
    return np.array(x_train), y_train2

In [7]:
#Function for windowing test signal, predictions are recieved 
#from the model in batches to limit memory issues
def windowtest(EMGtrain, spiketrain, window_size, model):
    
    EMGtrain = standardise(EMGtrain)
    x_train = []
    y_train = []
    predictions = []
    count = 1
    for i in range(30,EMGtrain.shape[0]-120):
        x_train.append(EMGtrain[i-10:i+(window_size-10),:])
        y_train.append(spiketrain[i, 0:5])
        if count%8162 == 0:
            predictions.append(model(np.array(x_train)))
            x_train = []
        count = count + 1
            
    y_train = np.array(y_train)
    y_train2 = []
    for i in range(5):
        y_train2.append(y_train[:,i])
    
    return tf.concat(predictions, axis = 1), y_train2

In [8]:
#Outputs a CNN model that recieves a HD-sEMG signal window as input and outputs
#a 0 or 1 label based on wether the respective MU it has been trained to detect
#is present in the signal. The number of these models in parallel can be given by MUs
def convolutional_model(input_shape, filter_num, filter_size, dense_num, MUs):
    input_signal = tf.keras.Input(shape = input_shape)
    out = []
    for i in range(1, MUs+1):
        X = tfl.Conv1D(filter_num, filter_size, activation = 'relu')(input_signal)
        X = tfl.Dropout(0.2)(X)
        X = tfl.Flatten()(X)
        X = tfl.Dense(dense_num, activation='relu')(X)
        X = tfl.Dropout(0.5)(X)
        output = tfl.Dense(1, activation = 'sigmoid', name='output_{}'.format(i))(X)
        out.append(output)
    
    model = tf.keras.Model(inputs = input_signal, outputs = out)
    return model

In [9]:
#Builds and compiles CNN model based on given parameters
def build_model(window_size, filter_num, filter_size, dense_num, MUs):
    model= convolutional_model((window_size, 192), filter_num, filter_size, dense_num, MUs)
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss = tf.keras.losses.BinaryCrossentropy(from_logits=False),
                  metrics = ['accuracy',RoA_m])
    return model

In [10]:
#Combines multiple 20s generated signals into a larger training set.
#Does not include the test set
def getFoldTrainingSet(test_fold, window_size):  
    #random.seed(60)
    training_folds = np.delete([1,2,3,4,5],test_fold-1)
    iter = False
    for fold in training_folds:
        EMGtrain = np.load('noise_data/30dB_fold{}_x.npy'.format(fold))
        spikes = np.load('noise_data/30dB_fold{}_y.npy'.format(fold))
        fold_windows , fold_spikes = windowtrain(EMGtrain, spikes, window_size)
        if iter == False:
            X = fold_windows
            Y = fold_spikes
        else:
            X = np.concatenate((X, fold_windows), axis = 0)
            for i in range(len(Y)):
                Y[i] = np.concatenate((Y[i],fold_spikes[i]))
        iter = True
    return X, Y

In [12]:
#Trains model and saves model that achieves best RoA on the validaiton set and the final model
def trainModel(model,X_train,Y_train, filter_num, filter_size, dense_num, MUs, test_fold):
    mc_vR= ModelCheckpoint('tuning_m/best_{}fnum_{}fsize_{}dnum_fold{}.h5'.format(filter_num, filter_size, dense_num, test_fold), monitor='val_RoA_m', mode='max', verbose=0, save_best_only=True)
    RoA_callback = AccuracyCallback('RoA_m')
    history = model.fit(X_train,
                        Y_train,
                        shuffle = True,
                        epochs = 100,
                        validation_split=0.2,
                        batch_size=256,
                        verbose = 0,
                        callbacks = [RoA_callback, mc_vR])
    model.save('tuning_m/final_{}fnum_{}fsize_{}dnum_fold{}.h5'.format(filter_num, filter_size, dense_num, test_fold))
    print('{}fnum_{}fsize_{}dnum_fold{} trained'.format(filter_num, filter_size, dense_num, test_fold))
    return None

In [ ]:
#Applied grid search training
window_size = 60
MUs = 5
random.seed(60)
for filter_num in [4,8,16]:
    for filter_size in [3,4,5]:
        for dense_num in [8, 16, 32]:
            dist = [1,2,3,4,5]
            for test_fold in dist:
                conv_model = build_model(window_size, filter_num, filter_size, dense_num, MUs)
                X_train, Y_train = getFoldTrainingSet(test_fold, window_size)
                trainModel(conv_model, X_train, Y_train, filter_num, filter_size, dense_num, MUs, test_fold)

In [14]:
for filter_num in [4,8,16]:
    for filter_size in [3,4,5]:
        for dense_num in [8, 16, 32]:
            RoAs = np.zeros(5)
            for fold in [1,2,3,4,5]:
                EMGtest=np.load('noise_data/30dB_fold{}_x.npy'.format(fold))
                spikes = np.load('noise_data/30dB_fold{}_y.npy'.format(fold))
                conv_model2 = load_model('tuning_m/best_{}fnum_{}fsize_{}dnum_fold{}.h5'.format(filter_num, filter_size, dense_num, fold), custom_objects={"RoA_m": RoA_m})
                predictions, Y = windowtest(EMGtest, spikes, window_size, conv_model2)
                RoAs[fold-1] = singleModelRoA(predictions, np.array(Y))
            print('{}fnum_{}fsize_{}dnum'.format(filter_num, filter_size, dense_num))
            print(np.mean(RoAs))

4fnum_3fsize_8dnum
0.971210014282331
4fnum_3fsize_16dnum
0.9734248631266661
4fnum_3fsize_32dnum
0.9662514039879815
4fnum_4fsize_8dnum
0.9766938692311449
4fnum_4fsize_16dnum
0.9773561553833996
4fnum_4fsize_32dnum
0.976391374801295
4fnum_5fsize_8dnum
0.971670870898594
4fnum_5fsize_16dnum
0.9732618753889971
4fnum_5fsize_32dnum
0.9677323794527327
8fnum_3fsize_8dnum
0.9676458748369878
8fnum_3fsize_16dnum
0.9706411795027157
8fnum_3fsize_32dnum
0.9572538371270722
8fnum_4fsize_8dnum
0.9707605948220914
8fnum_4fsize_16dnum
0.9754843720383987
8fnum_4fsize_32dnum
0.9710371806013047
8fnum_5fsize_8dnum
0.975552171612933
8fnum_5fsize_16dnum
0.9758203665344787
8fnum_5fsize_32dnum
0.9706037976453823
16fnum_3fsize_8dnum
0.9668947003520371
16fnum_3fsize_16dnum
0.9608222636047101
16fnum_3fsize_32dnum
0.9676458782186771
16fnum_4fsize_8dnum
0.9698732964574116
16fnum_4fsize_16dnum
0.9801148164561735
16fnum_4fsize_32dnum
0.9646112286017452
16fnum_5fsize_8dnum
0.9739285291609642
16fnum_5fsize_16dnum
0.97540189